# Chrome Built-in AI Demo in JupyterLab and JupyterLite Python Notebook

This notebook demonstrates the Python wrapper for Chrome's built-in AI Prompt API.

https://developer.chrome.com/docs/ai/prompt-api

**Requirements:**
- Chrome browser with Prompt API enabled
- Running in JupyterLab, Jupyter Notebook, or JupyterLite (statically served, all execution locally in browser)

## For developers: Using the Prompt API
* Open Chrome and go to `chrome://flags`.
* Enable the `#prompt-api-for-gemini-nano` flag.
* Enable the `#optimization-guide-on-device-model` flag. If you see a "BypassPerfRequirement" option, select it.
* Restart Chrome.

### &#128679; JupyterLab Only ATM &#128679;
The Python wiki3_ai package wrapping the Prompt API currently only works in JupyterLab notebooks, not JupyterLite.

In [2]:
%pip install -q anywidget ipywidgets wiki3_ai

Note: you may need to restart the kernel to use updated packages.


In [3]:
%gui asyncio

In [4]:
# Import the library
from wiki3_ai import LanguageModel

In [5]:
LanguageModel.widget()

In [6]:
import wiki3_ai
wiki3_ai.__version__

'0.5.1'

## Check Availability

First, let's check if the Prompt API is available in your browser.

In [7]:
# Check if the API is available
await LanguageModel.availability()

'available'

## Get Model Parameters

Let's see what parameters the model supports.

In [8]:
params = await LanguageModel.params()

if params:
    print(f"Default temperature: {params.get("defaultTemperature")}")
    print(f"Max temperature: {params.get("maxTemperature")}")
    print(f"Default top-K: {params.get("defaultTopK")}")
    print(f"Max top-K: {params.get("maxTopK")}")
else:
    print("Model parameters not available")

params

Default temperature: 1
Max temperature: 2
Default top-K: 3
Max top-K: 128


{'defaultTopK': 3,
 'maxTopK': 128,
 'defaultTemperature': 1,
 'maxTemperature': 2}

## Create LanguageModel Session

In [9]:
lm = await LanguageModel.create()
lm

## Simple Prompt

Create a session and send a simple prompt.

In [10]:
# Send a prompt
result = await lm.prompt("Write a haiku about Python programming.")
print(result)

Code flows, clear and bright,
Indentation guides the way,
Logic takes new form. 






## System Prompt

Use a system prompt to set the context for the conversation.

In [11]:
# Create a session with a system prompt
assistant_session = await LanguageModel.create(
    {
        "initialPrompts": [
            {
                "role": "system",
                "content": "You are a helpful Python programming assistant who gives concise answers.",
            }
        ]
    }
)

# Ask a question
result = await assistant_session.prompt("How do I read a CSV file in Python?")
print(result)

```python
import csv

with open('your_file.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        print(row)
```

Replace `'your_file.csv'` with the actual file name.  This opens the CSV, creates a reader object, and iterates through each row.  Each `row` is a list of strings.  For more complex CSVs, consider `csv.DictReader`.



In [12]:
result = await assistant_session.prompt("Please return just the code to read a CSV file in Python.  Include detailed comments.")
print(result)

```python
import csv

def read_csv(filename):
  """
  Reads a CSV file and prints each row.

  Args:
    filename: The name of the CSV file to read.
  """
  try:
    with open(filename, 'r', newline='') as file:  # Open the file in read mode ('r') using 'with' for automatic closing.  newline='' prevents extra blank rows.
      reader = csv.reader(file)  # Create a CSV reader object.
      for row in reader:  # Iterate over each row in the CSV file.
        print(row)  # Print the row.
  except FileNotFoundError:
    print(f"Error: File '{filename}' not found.")
  except Exception as e:
    print(f"An error occurred: {e}")

# Example usage:
#read_csv('your_file.csv')  # Replace 'your_file.csv' with your CSV file name
```


## Streaming Response

Stream the response as it's generated.

In [12]:
print("Response: ", end="")
async for chunk in assistant_session.prompt_streaming("Tell me a short joke about computers."):
    print(chunk, end="", flush=True)
print()

Response: Why did the computer get glasses? 

Because it needed to improve its website! 



In [13]:
async for chunk in assistant_session.prompt_streaming("Read me a long poem please."):
    print(chunk, end="", flush=True)
print()

Okay, here's a long poem I've crafted. It's about a journey through a fantastical, ever-changing landscape.  It’s meant to evoke imagery and a sense of wonder.  It's a bit abstract, but hopefully, you'll find something to enjoy!

**The Shifting Sands of Aethel**

From slumber deep, where starlight softly sleeps,
A traveler awoke, where twilight gently creeps.
No map he held, no compass to guide,
Just whispers carried on a wind-swept tide.
He stepped forth then, into Aethel's embrace,
A land of wonder, a timeless, mystic space.

The ground beneath, a shifting, golden sand,
That flowed and molded at a whim of hand.
Crystalline towers, kissed by amethyst skies,
Reflected hues in his questioning eyes.
The sun, a pearl, in a nebula's gleam,
Painted the landscape of a waking dream.

He journeyed onward, through forests of jade,
Where luminous flora in silent dances swayed.
Giant mushrooms, with caps of emerald light,
Cast shadows dancing in the endless night.
Creatures of myth, with wings of

## Multi-turn Conversation

Have a conversation with context maintained across prompts.

In [14]:
# First message
result1 = await assistant_session.prompt("What is a list comprehension?")
print("Assistant:", result1)
print()

Assistant: A list comprehension is a concise way to create new lists based on existing iterables (like lists, tuples, or ranges). It's a more compact and often faster alternative to using a `for` loop to build a list.

**Basic Syntax:**

```python
new_list = [expression for item in iterable if condition]
```

*   **`expression`**:  What you want to put into the new list. It can involve the `item`.
*   **`item`**:  Represents each element in the `iterable`.
*   **`iterable`**: The sequence (list, tuple, range, string, etc.) you're iterating over.
*   **`condition`** (optional):  A filter; only items that meet the condition will be included in the new list.



**Example:**

```python
# Create a list of squares of numbers from 0 to 9
squares = [x**2 for x in range(10)] 
print(squares)  # Output: [0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

# Create a list of even numbers from 0 to 19
even_numbers = [x for x in range(20) if x % 2 == 0]
print(even_numbers) # Output: [0, 2, 4, 6, 8, 10, 12, 14, 16,

In [15]:
# Follow-up message (the assistant remembers the context)
result2 = await assistant_session.prompt("Can you show me an example?")
print("Assistant:", result2)

Assistant: ```python
# Example: Create a list of the first 5 even numbers

# Without list comprehension (using a loop):
even_numbers_loop = []
for i in range(10):  # Iterate from 0 to 9
  if i % 2 == 0:  # Check if the number is even
    even_numbers_loop.append(i)

print(f"Even numbers (loop): {even_numbers_loop}")

# With list comprehension:
even_numbers_comprehension = [i for i in range(10) if i % 2 == 0]

print(f"Even numbers (comprehension): {even_numbers_comprehension}")
```

This code demonstrates how the same task (creating a list of even numbers) can be achieved using both a `for` loop and a list comprehension. The list comprehension is more concise and often considered more Pythonic. Both will produce the same output: `[0, 2, 4, 6, 8]`.


## Check Token Usage

Monitor how many tokens you've used.

In [16]:
print(f"Current usage: {assistant_session.input_usage}/{assistant_session.input_quota} tokens")

# Measure how many tokens a prompt would use
usage = await assistant_session.measure_input_usage("What is machine learning?")
print(f"\nThis prompt would use approximately {usage} tokens")

Current usage: 1826/9216 tokens

This prompt would use approximately 13 tokens


## Structured Output

Use JSON schema to get structured responses.

In [17]:
import json

# Define a JSON schema
schema = {
    "type": "object",
    "required": ["sentiment", "score"],
    "properties": {
        "sentiment": {"type": "string", "enum": ["positive", "negative", "neutral"]},
        "score": {"type": "number", "minimum": 0, "maximum": 1},
    },
}

# Get structured response
result = await lm.prompt(
    "Analyze this review: The product exceeded all my expectations! Absolutely amazing!",
    {"responseConstraint": schema},
)

# Parse the JSON response
data = json.loads(result)
print(f"Sentiment: {data['sentiment']}")
print(f"Score: {data['score']}")

Sentiment: positive
Score: 0.95


## Session Cloning

Clone a session to create different conversation branches.

In [18]:
# Start a story
story_session = await LanguageModel.create(
    {"initialPrompts": [{"role": "system", "content": "You are a creative storyteller."}]}
)

await story_session.prompt("Once upon a time, there was a dragon.")

# Create two different story branches
branch1 = await story_session.clone()
branch2 = await story_session.clone()

print("=== Branch 1 ===")
async for chunk in branch1.prompt_streaming("The dragon was friendly and helpful."):
    print(chunk, end="", flush=True)
print("=== End of 1 ===")

print("=== Branch 2 ===")
async for chunk in branch2.prompt_streaming("The dragon was fierce and terrifying."):
    print(chunk, end="", flush=True)
print("=== End of 2 ===")

=== Branch 1 ===
Once upon a time, there was a dragon named Zephyr. Now, Zephyr wasn't like the other dragons. Oh no. He didn't hoard gold or breathe scorching flames. Instead, he breathed bubbles. Big, iridescent bubbles that shimmered with all the colors of a rainbow. And he didn’t roar, he hummed. A low, melodic hum that resonated through the mountains and coaxed wildflowers to bloom even in the harshest winters.

He lived not in a dark, forbidding cave, but in a hidden valley filled with sparkling waterfalls and fields of glowing moss. He wasn’t fierce or intimidating; in fact, Zephyr was terribly shy. He preferred observing the tiny creatures of the valley – the bumble-winged beetles, the gossiping glow-worms, and the mischievous sprites who loved to tangle his tail in vines. He’d gently puff bubbles towards them, each one containing a fleeting image – a sunrise, a laughing child, a field of ripe berries. 

The villagers in the nearby hamlet of Oakhaven whispered stories of the “B

## Cleanup

Destroy sessions when you're done to free up resources.

In [19]:
await lm.destroy()
await assistant_session.destroy()
await story_session.destroy()
await branch1.destroy()
await branch2.destroy()

print("✅ All sessions destroyed")

✅ All sessions destroyed
